In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../metnet')
sys.path.append('../common')

In [3]:
from metnet import MetNet
import torch
import torch.nn.functional as F

import loss_utils

In [4]:
FORECAST = 10
model = MetNet(
        hidden_dim=32,
        forecast_steps=FORECAST, # should be 24 timesteps out
        input_channels=1, # 12 timeteps in
        output_channels=1, # 1 data channel in
        sat_channels=1, # 1 data channel in
        input_size=32, # =128/4, where 128 is the image dimensions
)

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [6]:
model = model.to(DEVICE)

In [7]:
# input_size = (1, 24, 1, 64, 64)
# torchinfo.summary(model, input_size)

In [8]:
sum(p.numel() for p in model.parameters())

1823649

In [9]:
BATCH_SIZE = 1

In [10]:
scaler = torch.cuda.amp.GradScaler()

In [11]:
x = torch.randn((BATCH_SIZE, 12, 1, 128, 128)).to(DEVICE)
y = torch.randn((BATCH_SIZE, FORECAST, 1, 64, 64), dtype=torch.float16).to(DEVICE)

In [12]:
criterion = loss_utils.MS_SSIMLoss(channels=FORECAST)

In [13]:
with torch.cuda.amp.autocast():
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    

/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [ ]:
scaler.scale(loss).backward()
scaler.step(optimizer)
scaler.update()

In [9]:
BATCH_SIZE = 1

In [10]:
# MetNet expects original HxW to be 4x the input size
# x = torch.randn((2, 12, 16, 128, 128))
x = torch.randn((BATCH_SIZE, 12, 1, 128, 128)).to(DEVICE)
y = torch.randn((1, FORECAST, 1, 64, 64)).to(DEVICE)
criterion = loss_utils.MS_SSIMLoss(channels=FORECAST)

out = model(x)
loss = criterion(out, y)
loss.backward()

out = model(x)
loss = criterion(out, y)
loss.backward()

/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [13]:
t1 = torch.cuda.get_device_properties(0).total_memory
r1 = torch.cuda.memory_reserved(0)
a1 = torch.cuda.memory_allocated(0)
f1 = r1-a1  # free inside reserved

In [14]:
t1, r1, a1, f1

(8370192384, 23068672, 8190976, 14877696)

In [16]:
a1

9206784

In [13]:
x = torch.randn((1, 12, 1, 128, 128)).to(DEVICE)
out = model(x)


In [14]:
t2 = torch.cuda.get_device_properties(0).total_memory
r2 = torch.cuda.memory_reserved(0)
a2 = torch.cuda.memory_allocated(0)
f2 = r2-a2  # free inside reserved

In [19]:
(a2 - a1) / a1

720.3252141029919

In [20]:
(f1 - f2)

682258432

In [ ]:
out.shape

In [11]:
# Modified MetNet creates predictions for the center 1/2
y = torch.randn((1, 10, 1, 64, 64)).to(DEVICE)

In [12]:
F.mse_loss(out, y).backward()